# Training on large dataset with attention model

After implementing [Beamsearch on a large dataset](BeamSearchOnLargeDataset.ipynb), I'll now add an attention model.
As trainings set I use the [European Parliament Proceedings Parallel Corpus 1996-2011](http://statmt.org/europarl/).

I first intented to implement it also with `Keras`. First of all, there is no built-in implementation of an attention layer or an attention decoder (it's planned atm). There are several projects like [keras-attention](https://github.com/datalogue/keras-attention) or a bit modified [monitonic-keras-attention](https://github.com/andreyzharkov/keras-monotonic-attention) (that works really better). Also there is [seq2seq project](https://github.com/farizrahman4u/seq2seq) that as lot of issues open. And a promising looking [NMT Keras](https://nmt-keras.readthedocs.io/en/latest/) that failed to install all dependency. I wouldn't mind a reimplemetation on my own (or improving one of these), just as I do the project anyway for learning purposes. After a while I really found this approach disturbing. I don't like switching around between the real high level layers of `Keras` down to `keras.backend` when I pretty much have to low level implement everything (in a different way to usual `Keras`) and in addition everything in object oriented extension of Layers, Cell and so on (with passing all parameters along, as GRU/LSTM have to be changed they also have to be reimplementend, vectorization with own time distributed layers, the masking layer won't work with further inputs like the weighted context vector, so we'll implement also Masking, ...). Just look into the projects and there is a lot of noisy code inside detracting from the original algorithm. 

Another idea is to use `tf.keras` instead of `Keras`. Look at the approach taken [here](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb?linkId=53292082#scrollTo=AOpGoE2T-YXS) - I found it after I've implemented this notebook, but on a first glance, it looks short and elegant. The advantage of using the tensorflow reimplementation `tf.keras` instead of `Keras` might be that we don't need to with `keras.backend` and can work directly on the computation graph with just the inputs and output of the `tf.keras.layers`. My suggestion would be that for new projects using `tf.keras` from the start can make things easier later.

Attention as basic idea is pretty simple: While decoding, we'll look back to weighted (what has to be learned) encoded states where the weights depend on the current decoding position, the previous (or current) hidden state of the decoder, and maybe to the previous alignment. We create a contect vector of them and use it also as an input (beside the last generated token) to the decoder. For performance we might only look to local hidden states around an alignment prediction (that can linear in the simplest form or usually als learnable). That's not so tough to represent as direct computation, but as we never work in Keras with the computation graph directly, it's harder than it should be. As technical detail, usually we don't take the context vector as another input (as we would have to combine inputs of different dimensions), but combine the context vector with the hidden state of the decoder to a modified hidden state. That's why we'd also change a lot in Keras.

With tensorflow we're closer to research and as I anyway intended to use multiple frameworks, I'll follow now the [seq2seq tutorial from tensorflow](https://www.tensorflow.org/tutorials/seq2seq). So, in this notebook there will be also a tensorflow implementation of the raw seq2seq model and Beam Search. I also refactored all the preparation work into a module.

Here, I will work with Luong Attention as it is the proposal from the tensorflow seq2seq tutorial and if I understand it right, it is very connected to dynamic neuronal network memories. I plan to investigate it later separate. There shouldn't be anyway much difference in results to Bahdanau and in case it's easy to switch to another attention mechnism in tensorflow (just change the class name). I didn't implement local allignments following a [discussion from a tensorflow feature request](https://github.com/tensorflow/tensorflow/issues/10842).

I still use a bidirectional encoder based on GRUs and a decoder network with GRUs with the same hyperparameters as in [BeamSearchOnLargeDataset with Keras](BeamSearchOnLargeDataset.ipynb) used. I don't use much of the most modern features of tensorflow (eager execution or tensorflow.data API or flags for hyperparameters or plug in summaries and use tensorbard in addition). I intend to try them out later. Here, I tried to work with as little as possible magic.

In [21]:
# Check that there is a GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14165716698337561091
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7692746752
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14130295096720600696
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
# Check that Cuda/Cudnn/GPU works as intended
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.layers import core as layers_core
from tqdm import tqdm_notebook as tqdm

from utils.download import download_and_extract_resources
from utils.linguistic import bleu_scores_europarl, preprocess_input_europarl as preprocess
from utils.preparation import Europarl, RANDOM_STATE

Fixed random seed to 42


Using TensorFlow backend.


In [4]:
MAX_INPUT_LENGTH = 100
MAX_TARGET_LENGTH = 125
LATENT_DIM =  256  # was 512, but we should be able to use a smaller hidden representation as we are looking back anyway as needed
EPOCHS = 20
BATCH_SIZE = 128  # was 64, but tensorflow implementation doesn't need so much GPU memory so can increase batch size
DROPOUT = 0.25  # Dropout on input and output for the RNN cells, so effective dropout is 0.5, but works slightly better so
TEST_SIZE = 2500
BEAM_WIDTH = 5
EMBEDDING_TRAINABLE = True  # Improves results significant and for at least it's not the most dominant training time factor (that's the output softmax layer)

## Download and explore data

In [5]:
europarl = Europarl()
download_and_extract_resources(fnames_and_urls=europarl.external_resources, dest_path=europarl.path)

de-en.tgz already downloaded (188.6 MB)
en.wiki.bpe.op5000.model already downloaded (0.3 MB)
en.wiki.bpe.op5000.d300.w2v.bin.tar.gz already downloaded (6.2 MB)
de.wiki.bpe.op5000.model already downloaded (0.3 MB)
de.wiki.bpe.op5000.d300.w2v.bin.tar.gz already downloaded (5.7 MB)


In [6]:
europarl.load_and_preprocess(max_input_length=MAX_INPUT_LENGTH, max_target_length=MAX_TARGET_LENGTH)

Total number of unfiltered translations 1920209
Filtered translations with length between (1, input=100/target=125) characters: 597331


In [7]:
europarl.df.head()

,input_texts,target_texts,input_length,target_length,input_sequences,target_sequences
0,resumption of the session,wiederaufnahme der sitzungsperiode,25,34,"[1, 344, 146, 498, 90, 6, 3, 3235, 90, 2]","[1, 247, 351, 750, 5, 934, 43, 3158, 4762, 2]"
5,"please rise, then, for this minute' s silence.","ich bitte sie, sich zu einer schweigeminute zu...",46,55,"[1, 3005, 416, 77, 359, 4, 241, 4, 17, 76, 451...","[1, 241, 156, 72, 3112, 54, 4, 39, 26, 95, 473..."
6,(the house rose and observed a minute' s silence),(das parlament erhebt sich zu einer schweigemi...,49,52,"[1, 29, 140, 414, 3231, 8, 3106, 2484, 9, 451,...","[1, 35, 2444, 2269, 2109, 625, 39, 26, 95, 473..."
7,"madam president, on a point of order.","frau präsidentin, zur geschäftsordnung.",37,39,"[1, 1599, 134, 546, 4, 19, 9, 918, 6, 535, 5, 2]","[1, 1161, 2266, 52, 4, 132, 2232, 1516, 3, 2]"
12,"if the house agrees, i shall do as mr evans ha...","wenn das haus damit einverstanden ist, werde i...",58,86,"[1, 530, 3, 414, 1434, 35, 4, 305, 186, 321, 3...","[1, 835, 19, 684, 494, 21, 161, 48, 838, 30, 4..."


In [8]:
print("English subwords", europarl.bpe_input.sentencepiece.EncodeAsPieces("this is a test for pretrained bytepairembeddings"))
print("German subwords", europarl.bpe_target.sentencepiece.EncodeAsPieces("das ist ein test für vortrainierte zeichengruppen"))

English subwords ['▁this', '▁is', '▁a', '▁test', '▁for', '▁pre', 'tr', 'ained', '▁by', 'te', 'pa', 'ire', 'm', 'bed', 'd', 'ings']
German subwords ['▁das', '▁ist', '▁ein', '▁test', '▁für', '▁v', 'ort', 'rain', 'ierte', '▁zeich', 'eng', 'ruppen']


In [9]:
# Those will be the inputs for the seq2seq model (that needs to know how long the sequences can get)
max_len_input = europarl.df.input_sequences.apply(len).max()
max_len_target = europarl.df.target_sequences.apply(len).max()
(max_len_input, max_len_target)

(52, 71)

In [10]:
train_ids, val_ids = train_test_split(np.arange(europarl.df.shape[0]), test_size=0.1, random_state=RANDOM_STATE)  # fixed random_state

In [11]:
tf.reset_default_graph()

with tf.device('/gpu:0'):

    encoder_inputs = tf.placeholder(
        shape=(None, None),  # batch_size x max_len_input
        dtype=tf.int32,
        name='encoder_inputs' 
    )
    batch_size = tf.shape(encoder_inputs)[0]
    beam_width = tf.placeholder_with_default(1, shape=[])
    dropout = tf.placeholder_with_default(tf.cast(0.0, tf.float32), shape=[])
    keep_prob = tf.cast(1.0, tf.float32) - dropout

    embedding_encoder = tf.get_variable(
        "embedding_encoder", 
        initializer=tf.constant(europarl.bpe_input.embedding_matrix),
        trainable=EMBEDDING_TRAINABLE,
    )
    encoder_emb_inp = tf.nn.embedding_lookup(
        embedding_encoder,
        encoder_inputs,
        name="encoder_emb_inp"
    )
    
    input_sequence_length = tf.placeholder(
        shape=(None, ),
        dtype=tf.int32,
        name='input_sequence_length'
    )
    
    rnn_cell_type = tf.nn.rnn_cell.GRUCell
    encoder_forward_cell = tf.nn.rnn_cell.DropoutWrapper(
        rnn_cell_type(num_units=LATENT_DIM // 2, name='encoder_forward_cell'),
        input_keep_prob=keep_prob,
        output_keep_prob=keep_prob,  # state_keep_prob not set as it was not helpful here
        dtype=tf.float32,
    )
    encoder_backward_cell = tf.nn.rnn_cell.DropoutWrapper(
        rnn_cell_type(num_units=LATENT_DIM // 2, name='encoder_backward_cell'),
        input_keep_prob=keep_prob,
        output_keep_prob=keep_prob,
        dtype=tf.float32,
    )
    encoder_bi_outputs, encoder_bi_state = tf.nn.bidirectional_dynamic_rnn(
        encoder_forward_cell, encoder_backward_cell,
        inputs=encoder_emb_inp,
        sequence_length=input_sequence_length,
        time_major=False,
        dtype=tf.float32,
    )
    encoder_outputs = tf.concat(encoder_bi_outputs, -1)
    encoder_state = tf.concat(encoder_bi_state, -1)
    
    # Regarding time_major:
    # If true, these `Tensors` must be shaped `[max_time, batch_size, depth]`.
    # If false, these `Tensors` must be shaped `[batch_size, max_time, depth]`.
    # Using `time_major = True` is a bit more efficient because it avoids
    # transposes at the beginning and end of the RNN calculation.  However,
    # most TensorFlow data is batch-major, so by default this function
    # accepts input and emits output in batch-major form.
    #
    # for simplicity I work with batch major here instead of time_major
    # so I don't need to transpose inputs and transpose back for attention mechanism
    
    decoder_inputs = tf.placeholder(
        shape=(None, None),  # batch_size x max_len_target
        dtype=tf.int32,
        name='decoder_inputs' 
    )
    embedding_decoder = tf.get_variable(
        "embedding_decoder", 
        initializer=tf.constant(europarl.bpe_target.embedding_matrix),
        trainable=EMBEDDING_TRAINABLE,
    )
    decoder_emb_inp = tf.nn.embedding_lookup(
        embedding_decoder,
        decoder_inputs,
        name="decoder_emb_inp"
    )
    
    target_sequence_length = tf.placeholder(
        shape=(None, ),
        dtype=tf.int32,
        name='target_sequence_length'
    )
    
    # tiling is necessary to work with BeamSearchDecoder
    # read carefully the NOTE on constructor in
    # https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/AttentionWrapper 
    tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(encoder_outputs, multiplier=beam_width)
    tiled_encoder_state = tf.contrib.seq2seq.tile_batch(encoder_state, multiplier=beam_width)
    tiled_sequence_length = tf.contrib.seq2seq.tile_batch(input_sequence_length, multiplier=beam_width)
    
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        LATENT_DIM,
        memory=tiled_encoder_outputs,
        memory_sequence_length=tiled_sequence_length,
        dtype=tf.float32,
        name='attention_mechanism',
    )
    decoder_rnn_cell = tf.nn.rnn_cell.DropoutWrapper(
        rnn_cell_type(num_units=LATENT_DIM, name='decoder_cell'),
        input_keep_prob=keep_prob,
        output_keep_prob=keep_prob,
        dtype=tf.float32,
    )
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        decoder_rnn_cell,
        attention_mechanism,
        attention_layer_size=LATENT_DIM, 
        name='attention_wrapper',
    )

    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=decoder_emb_inp, 
        sequence_length=target_sequence_length,
        time_major=False,
        name="decoder_training_helper",
    )
    
    projection_layer = layers_core.Dense(
        units=len(europarl.bpe_target.tokens),
        use_bias=False,
        name='projection_layer',
    )
    
    initial_state=decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size).clone(
        cell_state=encoder_state
    )
    decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=decoder_cell,
        helper=training_helper,
        initial_state=initial_state,
        output_layer=projection_layer,
    )
    outputs, _final_state, _final_sequence_length = tf.contrib.seq2seq.dynamic_decode(  
        decoder,
        output_time_major=False,
        impute_finished=False,
    )
    logits = outputs.rnn_output
    
    decoder_outputs = tf.placeholder(
        shape=(None, None),  # batch_size x max_len_target
        dtype=tf.int32,
        name='decoder_outputs',
    )
    target_weights = tf.cast(tf.sequence_mask(target_sequence_length), dtype=tf.float32)
    train_loss = tf.contrib.seq2seq.sequence_loss(logits, decoder_outputs, target_weights)

    params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(
        t_list=gradients,
        clip_norm=1.,
    )
    
    optimizer = tf.train.AdamOptimizer()
    update_step = optimizer.apply_gradients(zip(clipped_gradients, params))
    
    inference_decoder_initial_state = decoder_cell.zero_state(
        dtype=tf.float32,
        batch_size=batch_size * beam_width  # tricky and somehow unintuitive, but necessary
    ).clone(
        cell_state=tiled_encoder_state
    )
    inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
        cell=decoder_cell,
        embedding=embedding_decoder,
        start_tokens=tf.fill([batch_size], europarl.bpe_target.start_token_idx),
        end_token=europarl.bpe_target.stop_token_idx,
        initial_state=inference_decoder_initial_state,
        beam_width=BEAM_WIDTH,
        output_layer=projection_layer,
        length_penalty_weight=1.0,  # https://machinelearningmastery.com/configure-encoder-decoder-model-neural-machine-translation/
    )

    
    inference_outputs, _inference_final_state, _inference_final_sequence_length = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder,
        maximum_iterations=tf.round(tf.reduce_max(input_sequence_length) * 2),  # a bit more flexible than max_len_target
        impute_finished=False,
    )

In [12]:
def run_train_batch(batch_ids):
    batch_input_sequences = europarl.df.input_sequences.iloc[batch_ids]
    batch_input_lengths = batch_input_sequences.apply(len)
    batch_target_sequences = europarl.df.target_sequences.iloc[batch_ids]
    batch_target_lengths = batch_target_sequences.apply(len) - 1

    batch_input_padded = tf.keras.preprocessing.sequence.pad_sequences(
        batch_input_sequences,
        maxlen=max_len_input,
        dtype=int,
        padding='post'
    )
    batch_target_padded = tf.keras.preprocessing.sequence.pad_sequences(
        batch_target_sequences,
        maxlen=max_len_target,
        dtype=int,
        padding='post'
    )
    pred, loss, _ = sess.run(
        fetches=[
            outputs, train_loss, update_step
        ],
        feed_dict={
            encoder_inputs: batch_input_padded,
            input_sequence_length: np.array(batch_input_lengths),
            decoder_inputs: batch_target_padded[:, :batch_target_lengths.max()],
            target_sequence_length: np.array(batch_target_lengths),
            decoder_outputs: batch_target_padded[:, 1:batch_target_lengths.max() + 1],
            dropout: DROPOUT,
        }
    )
    return loss

def run_val_batch(batch_ids):
    batch_input_sequences = europarl.df.input_sequences.iloc[batch_ids]
    batch_input_lengths = batch_input_sequences.apply(len)
    batch_target_sequences = europarl.df.target_sequences.iloc[batch_ids]
    batch_target_lengths = batch_target_sequences.apply(len) - 1

    batch_input_padded = tf.keras.preprocessing.sequence.pad_sequences(
        batch_input_sequences,
        maxlen=max_len_input,
        dtype=int,
        padding='post'
    )
    batch_target_padded = tf.keras.preprocessing.sequence.pad_sequences(
        batch_target_sequences,
        maxlen=max_len_target,
        dtype=int,
        padding='post'
    )
    loss = sess.run(
        fetches=[train_loss],
        feed_dict={
            encoder_inputs: batch_input_padded,
            input_sequence_length: np.array(batch_input_lengths),
            decoder_inputs: batch_target_padded[:, :batch_target_lengths.max()],
            target_sequence_length: np.array(batch_target_lengths),
            decoder_outputs: batch_target_padded[:, 1:batch_target_lengths.max() + 1],
        }
    )
    return loss

def run_validation_loss():
    return np.mean([
        run_val_batch(ids)
        for ids 
        in np.array_split(val_ids, np.ceil(len(val_ids) / BATCH_SIZE))
    ])

In [13]:
config = tf.ConfigProto(
    allow_soft_placement=True,  # needed as recommendation from https://github.com/tensorflow/tensorflow/issues/2292
    log_device_placement=True,
)
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

batches_per_epoch = np.ceil(len(train_ids) / BATCH_SIZE)
for epoch in range(EPOCHS):
    shuffled_ids = np.random.permutation(train_ids)
    batch_splits = np.array_split(shuffled_ids, batches_per_epoch)
    train_losses = []
    N = len(batch_splits)
    with tqdm(batch_splits, desc=f"Epoch {epoch+1}") as t:
        for train_batch_ids in t:
            batch_loss = run_train_batch(train_batch_ids)
            train_losses.append(batch_loss)
            t.set_postfix(train_loss=np.mean(train_losses))
        print("train_loss", np.mean(train_losses), "val_loss", run_validation_loss())
        
validation_input_sequences = europarl.df.input_sequences.iloc[val_ids[:BATCH_SIZE]]
validation_input_lengths = validation_input_sequences.apply(len)

validation_input_padded = tf.keras.preprocessing.sequence.pad_sequences(
    validation_input_sequences,
    maxlen=max_len_input,
    dtype=int,
    padding='post'
)


train_loss 3.2934992 val_loss 2.3617246



train_loss 2.5020657 val_loss 2.1203256



train_loss 2.3381367 val_loss 2.0130618



train_loss 2.2496881 val_loss 1.9461743



train_loss 2.1938696 val_loss 1.9016857


...


train_loss 1.8547937 val_loss 1.6346791



train_loss 1.8338715 val_loss 1.6228687



train_loss 1.8179969 val_loss 1.608734



train_loss 1.8050221 val_loss 1.6028578



train_loss 1.7941648 val_loss 1.5963621


In [14]:
def predict(sentence):
    sequenced = europarl.bpe_input.subword_indices(preprocess(sentence))
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        [sequenced],
        maxlen=max_len_input,
        dtype=int,
        padding='post'
    )
    
    beam_search_output = sess.run(
        fetches=[inference_outputs],
        feed_dict={
            encoder_inputs: padded,
            input_sequence_length: [len(sequenced)],
            beam_width: BEAM_WIDTH,
        }
    )[0]
    
    return europarl.bpe_target.sentencepiece.DecodePieces([
        europarl.bpe_target.tokens[idx] for idx in beam_search_output.predicted_ids[0, :, 0].tolist()
    ])


In [15]:
name = 'tfattentionmodel'

saver = tf.train.Saver()
saver.save(sess, f"data/{name}.ckpt")
# tfattentionmodel.ckpt.index https://drive.google.com/open?id=1Xv2Qc9Gnac_Of9bIpQef5LZJnX9ij933
# tfattentionmodel.cpkt.meta https://drive.google.com/open?id=162WY8XEjyqfvitiIBfmq9oHLC1rQhyvr
# tfattentionmodel.cpkt.data-00000-of-00001 https://drive.google.com/open?id=1racW0agvk5nJ_xBaxifAaEWM8T9ot7FL

'data/tfattentionmodel.ckpt'

In [16]:
europarl.df.head()

,input_texts,target_texts,input_length,target_length,input_sequences,target_sequences
0,resumption of the session,wiederaufnahme der sitzungsperiode,25,34,"[1, 344, 146, 498, 90, 6, 3, 3235, 90, 2]","[1, 247, 351, 750, 5, 934, 43, 3158, 4762, 2]"
5,"please rise, then, for this minute' s silence.","ich bitte sie, sich zu einer schweigeminute zu...",46,55,"[1, 3005, 416, 77, 359, 4, 241, 4, 17, 76, 451...","[1, 241, 156, 72, 3112, 54, 4, 39, 26, 95, 473..."
6,(the house rose and observed a minute' s silence),(das parlament erhebt sich zu einer schweigemi...,49,52,"[1, 29, 140, 414, 3231, 8, 3106, 2484, 9, 451,...","[1, 35, 2444, 2269, 2109, 625, 39, 26, 95, 473..."
7,"madam president, on a point of order.","frau präsidentin, zur geschäftsordnung.",37,39,"[1, 1599, 134, 546, 4, 19, 9, 918, 6, 535, 5, 2]","[1, 1161, 2266, 52, 4, 132, 2232, 1516, 3, 2]"
12,"if the house agrees, i shall do as mr evans ha...","wenn das haus damit einverstanden ist, werde i...",58,86,"[1, 530, 3, 414, 1434, 35, 4, 305, 186, 321, 3...","[1, 835, 19, 684, 494, 21, 161, 48, 838, 30, 4..."


In [17]:
# Performance on some examples:
EXAMPLES = [
    'Hello.',
    'You are welcome.',
    'How do you do?',
    'I hate mondays.',
    'I am a programmer.',
    'Data is the new oil.',
    'It could be worse.',
    "I am on top of it.",
    "N° Uno",
    "Awesome!",
    "Put your feet up!",
    "From the start till the end!",
    "From dusk till dawn.",
]
for en in [sentence + '\n' for sentence in EXAMPLES]:
    print(f"{preprocess(en)!r} --> {predict(en)!r}")

'hello.' --> 'helfen sie.'
'you are welcome.' --> 'sie sind begrüßenswert.'
'how do you do?' --> 'wie sieht sie tun?'
'i hate mondays.' --> 'ich habe meine frage.'
'i am a programmer.' --> 'ich bin ein programm.'
'data is the new oil.' --> 'die daten ist das neue öl.'
'it could be worse.' --> 'es könnte schlimmer sein.'
'i am on top of it.' --> 'ich bin davon überzeugt.'
'n° uno' --> 'nitreo'
'awesome!' --> 'einwände!'
'put your feet up!' --> 'stellen sie ihre füße!'
'from the start till the end!' --> 'aus dem beginn des beginns!'
'from dusk till dawn.' --> 'von duskulanz.'


In [18]:
# Performance on training set:
for en, de in europarl.df[['input_texts', 'target_texts']][1:20].values.tolist():
    print(f"Original {en!r}, got {predict(en)!r}, exp: {de!r}")

Original "please rise, then, for this minute' s silence.", got 'ich bitte sie, diese schweigeminute zu sagen.', exp: 'ich bitte sie, sich zu einer schweigeminute zu erheben.'
Original "(the house rose and observed a minute' s silence)", got '(das parlament erhebt sich zu einer schweigeminute.)', exp: '(das parlament erhebt sich zu einer schweigeminute.)'
Original 'madam president, on a point of order.', got 'frau präsidentin, zur geschäftsordnung.', exp: 'frau präsidentin, zur geschäftsordnung.'
Original 'if the house agrees, i shall do as mr evans has suggested.', got 'wenn das haus einverstanden ist, werde ich herrn evans vorgeschlagen.', exp: 'wenn das haus damit einverstanden ist, werde ich dem vorschlag von herrn evans folgen.'
Original 'madam president, on a point of order.', got 'frau präsidentin, zur geschäftsordnung.', exp: 'frau präsidentin, zur geschäftsordnung.'
Original 'i would like your advice about rule 0 concerning inadmissibility.', got 'ich möchte gerne ihre ratspräs

In [19]:
# Performance on validation set
val_df = europarl.df.iloc[val_ids]
for en, de in val_df[['input_texts', 'target_texts']][1:20].values.tolist():
    print(f"Original {en!r}, got {predict(en)!r}, exp: {de!r}")

Original 'we congratulate you on a job very well done.', got 'wir beglückwünschen ihnen zu einer arbeit sehr gut.', exp: 'wir gratulieren dir zu deiner hervorragenden arbeit.'
Original 'in this case, i strongly disagree with what was said by the previous speaker, carl schlyter.', got 'in diesem fall stimme ich mit dem vorredner mit dem vorredner mit dem vorredner nicht zu.', exp: 'in diesem fall widerspreche ich klar dem, was der vorredner, carl schlyter, gesagt hat.'
Original 'it only makes sense to rebuild these if the refugees who fled are coming back.', got 'es ist nur sinnvoll, dies zu wiederaufbauen, wenn die flüchtlinge zurückkehren.', exp: 'deren wiederaufbau ist nur dann von nutzen, wenn die flüchtlinge aus den betreffenden gebieten wieder zurückkehren.'
Original 'eba: everything but arms.', got 'eba: alles, aber waffen.', exp: 'eba: everything but arms.'
Original 'in wider terms, this directive is not ambitious enough.', got 'in diesem sinne ist diese richtlinie nicht ehrgeiz

In [20]:
bleu = bleu_scores_europarl(
    input_texts=europarl.df.input_texts.iloc[val_ids[:TEST_SIZE]],
    target_texts=europarl.df.target_texts.iloc[val_ids[:TEST_SIZE]],
    predict=lambda text: predict(text)
)
print(f'average BLEU on test set = {bleu.mean()}')


average BLEU on test set = 0.23681209357919392


# Conclusion

The translations for the train and validations are decent. The main problem without attention was that the rnn decoder repeated itself and got confused what it intended to say (like "leider geht es nur um die realität und die realität.") The bleu score also improved from 0.183 to 0.237.

Now, the main problems are any words out of the vocabulary (meaning not seen often enough in the training set). Some of the problems will go away if we use the whole training data set instead of a subset. But in the long run, we might need a copy mechanism, a look up method and/or a dynamic neural network memory. Also, some translations make grammatical mistakes. I can imagine that multi learning (like also learn grammatical features or entity recognition maybe in combination with some data augmentation) would help. It's also somehow funny to see how accurately the translations are for complicated sentences from the validation set, but for my very simple own examples that are different to europarlament discussion, the translations are terrible.

Regarding tensorflow, allthough the code is a bit longer, it doesn't need much energy to figure what's going on. Every line manipulates the computational graph without much surprises. Allthough, debugging is still a hassle (cryptic error messages and no direct evaluation), it's not so mind blogging like working on different abstraction levels with Keras IMHO. It's also nice to see that a lot state-of-the art technology from research is already implemented in tensorflow. The computational speed went up here a lot, too. I think the main reason is that the tensorflow implementation only trains till end-of-sentence and not beyond (where Keras implementation masked them only to ignore in loss function). 

My next step will be to train the neural machine translation on the whole europarliament dataset (2.3 Mio > 600k) and also train for a longer time (even here the training did not converge after 20 epochs).